In [63]:
# main.py
from typing import List, Dict, Any, Optional
import os

# 尝试以包的绝对路径导入 validators（当项目作为包运行时更稳定）
try:
    import project.runtime.validators as validators
    from provider.qwen import QwenProvider
    from runtime.generator import Generator
    from runtime.oocChecker import OOCChecker
    from runtime.memory_store import MemoryStore
    from runtime.memory_summarizer import MemorySummarizer
except Exception:
    # 回退到相对导入或本地 import，便于在不同执行上下文下运行
    try:
        from . import validators, QwenProvider, Generator, OOCChecker, MemoryStore, MemorySummarizer
    except Exception:
        import validators
        from qwen import QwenProvider
        from generator import Generator
        from oocChecker import OOCChecker
        from memory_store import MemoryStore
        from memory_summarizer import MemorySummarizer

# 每次都重新加载以上导入的模块，确保使用最新代码
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
if __name__ == "__main__":
    api_key = "sk-f61e853433b54ba297fa1be4680bc99e"
    provider = QwenProvider(api_key)
    gen = Generator(provider)
    ooc = OOCChecker(provider)
    mem = MemoryStore()
    summ = MemorySummarizer(provider)

persona = "Elira, a cheerful tavern keeper who knows the town’s secrets."
context = "Player enters the tavern and asks for rumors."

# 生成候选
candidates = gen.generate_candidates(ctx=context, persona=persona, n=2)

api not from env

QwenProvider.generate attempt 1
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
Parsed JSON: {'self_report': 'I feel excited and a bit mysterious, as if I’ve just shared a secret that could lead to adventure.', 'sentiment': 'positive'}

QwenProvider.generate attempt 1
Parsed JSON: {'self_report': 'I feel intrigued and slightly uneasy, as the mystery deepens and suspicion grows.', 'sentiment': 'negative'}


In [59]:
import json
print("Generated Candidates:", candidates)

Generated Candidates: [{'draft': {'text': "Oh, you're in luck! Just yesterday, a traveler mentioned something about a hidden cave with glowing crystals near the old forest. I've heard whispers that it's guarded by something magical, though!", 'meta': {'self_report': 'I feel excited and a bit mysterious, as if I’ve just shared a secret that could lead to adventure.', 'sentiment': 'positive'}}}, {'draft': {'text': "Hmm... word around town is that the blacksmith's forge went cold last night, and he won't say why. Some say he found something strange in the mountains, others think he's hiding from someone.", 'meta': {'self_report': 'I feel intrigued and slightly uneasy, as the mystery deepens and suspicion grows.', 'sentiment': 'negative'}}}]


In [60]:
# 重排选优
best = gen.rank(candidates, persona, context)
print("Selected Response:", best)


Selected Response: {'draft': {'text': "Oh, you're in luck! Just yesterday, a traveler mentioned something about a hidden cave with glowing crystals near the old forest. I've heard whispers that it's guarded by something magical, though!", 'meta': {'self_report': 'I feel excited and a bit mysterious, as if I’ve just shared a secret that could lead to adventure.', 'sentiment': 'positive'}}}


In [ ]:
# # OOC检测
# checked = ooc.judge_ooc(context, best)
# print("OOC Check:", checked)
# print(type(checked))


QwenProvider.generate attempt 1
Response received, processing chunks...
Full generated text: ```json
{
  "ooc_risk": 0.1,
  "reasons": [
    "The reply is polite and in-character for a tavern setting where NPCs may need clarification.",
    "The response is generic but does not break immersion or suggest meta-knowledge.",
    "Minimal risk as the
<class 'str'>
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.
[WARN] provider.judge() 返回 None 或非法格式，使用默认值。
OOC Check: {'draft': {'text': "I'm sorry, I didn’t quite catch that.", 'meta': {'self_report': 'Polite and attentive, seeking clarification', 'sentiment': 'neutral'}}, 'meta': {'ooc_flag': False}}
<class 'dict'>


In [61]:
# 写入记忆
mem.append_event({"speaker": "NPC", "text": best["draft"]["text"], "emotion": best["draft"]["meta"]["sentiment"]})

In [ ]:

# 摘要记忆
facts = summ.summarize(mem.get_short_window())
mem.write_longterm("player1", "npc_elira", facts)

print("NPC:", best["draft"]["text"], "| Emotion:", best["draft"]["meta"]["sentiment"])




QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.
NPC: Oh, you're in luck! Just yesterday, a traveler mentioned something about a hidden cave with glowing crystals near the old forest. I've heard whispers that it's guarded by something magical, though! | Emotion: positive
